# Image Processing Laboratory Notebooks
---

This Jupyter Notebook is part of a series of computer laboratories that are designed
to teach image-processing programming; they are running on the EPFL's Noto server. They are the practical complement of the theoretical lectures of the EPFL's Master course 
[**MICRO-511 Image Processing I**](https://moodle.epfl.ch/course/view.php?id=522) taught by Prof. M. Unser and Prof. D. Van de Ville.

The project is funded by the Center for Digital Education and the School of Engineering. It is owned by the [Biomedical Imaging Group](http://bigwww.epfl.ch/). 
The distribution or reproduction of the notebook is strictly prohibited without the written consent of the authors.  &copy; EPFL 2025.

**Authors**:
    [Pol del Aguila Pla](mailto:pol.delaguilapla@epfl.ch), 
    [Kay Lächler](mailto:kay.lachler@epfl.ch),
    [Alejandro Noguerón Arámburu](mailto:alejandro.nogueronaramburu@epfl.ch), and
    [Daniel Sage](mailto:daniel.sage@epfl.ch).
    
---
# Lab 2.1: Filtering (9 Points)
**Released**: Thursday, November 6, 2025

**Submission deadline**: Monday, November 17, 2025, before 23:59 on [Moodle](https://moodle.epfl.ch/course/view.php?id=522)

**Grade weight**: Lab 2 (16 points), 10% of the overall grade

**Related lectures**: Chapter 3

### Student Name: 
### SCIPER: 

Double-click on this cell and fill your name and SCIPER number. Then, run the cell below to verify your identity in Noto and set the seed for random results.

In [ ]:
%use sos
import getpass
# This line recovers your camipro number to mark the images with your ID
uid = int(getpass.getuser().split('-')[2]) if len(getpass.getuser().split('-')) > 2 else ord(getpass.getuser()[0])
print(f'SCIPER: {uid}')

### Setup
In the next cell we import the Python libraries that we will use throughout the lab, as well as the `ImageViewer` class (Python package developed specifically for these laboratories, see documentation [here](https://github.com/Biomedical-Imaging-Group/interactive-kit/wiki/Image-Viewer), or run the python command `help(viewer)` after loading the class):

* [`matplotlib.pyplot`](https://matplotlib.org), to display images
* [`ipywidgets`](https://ipywidgets.readthedocs.io/en/latest/), to make the image display interactive
* [`numpy`](https://numpy.org/doc/stable/reference/index.html), for mathematical operations on arrays
* [`openCV` (cv2)](https://docs.opencv.org/2.4/index.html), for image-processing tasks
* [`scipy.ndimage`](https://docs.scipy.org/doc/scipy/reference/ndimage.html), Scipy's specific module for multidimensional image processing

Finally, we load the images you will use in the exercise to test your algorithms. 

In [ ]:
%use sos
# Configure plotting as dynamic
%matplotlib widget

# Import standard required packages for this exercise
import matplotlib.pyplot as plt
import numpy as np
import cv2 as cv 
import scipy.ndimage as ndi
import ipywidgets as widgets
from interactive_kit import imviewer as viewer

# Load images to be used in this exercise 
bikesgray = cv.imread('images/bikesgray.tif', cv.IMREAD_UNCHANGED).astype('float64')
camera = cv.imread('images/camera-16bits.tif', cv.IMREAD_UNCHANGED).astype('float64')
spots = cv.imread('images/spots.tif', cv.IMREAD_UNCHANGED).astype('float64')

Now we will import the `ImageAccess` class. You can find the documentation of the class [here](https://biomedical-imaging-group.github.io/image-access/). Moreover, we will put the images we will be using in the JavaScript kernel.

In [ ]:
%use javascript
%get bikesgray camera spots
// import ImageAccess as Image
var Image = require('image-access')

In this lab, you will review the basics of digital image filtering. You will review how to implement a convolution (the main ingredient of filtering) and explore topics such as separable filters, non-separable filters, filter design, and boundary conditions. The goal is for you to understand the filtering algorithm at a pixel-by-pixel level (implementing it in JavaScript) and, simultaneously, to learn the standard tools for using filtering in professional image-processing libraries in Python.

**Important:** Each cell that contains code begins with `%use sos` or `%use javascript`. This indicates if the code in this specific cell should be written in Python or JavaScript. Do not change or remove any lines of code that begin with a %. They are used for the notebook to run smoothly with `SoS` and need to be on the first line of each cell!

Good luck and enjoy! 

Get familiar with the images you will be using. Remember to browse through the images with the buttons `Next` & `Prev`, and pay attention to the histograms. 

In [ ]:
%use sos
# Declare image_list for ImageViewer
image_list = [bikesgray, camera, spots]

imgs_viewer = viewer(image_list, widgets=True, hist=True)

## 1. Edge detection (3 points)

In this section of the lab, we will review one of the most fundamental image processing operations: edge detection. Moreover, we will use this filter to review the concepts of convolution and correlation. As you may recall from the course, edge detection algorithms aim to identify steep intensity differences between neighboring pixels. A standard horizontal-edge detector (detecting changes in the $y$ direction) is the filter represented by the impulse response

$$h[m,n] = 
\begin{bmatrix}
    -1 & -2 & -1 \\
    0 & 0 & 0 \\
    1 & 2 & 1
\end{bmatrix}\,.$$

If pixels neighbouring a given pixel position all have a similar value, the output at that pixel will have a value close to zero after applying the mask. On the other hand, if there is a big difference in the values of the top and bottom rows, this likely indicates some horizontal edge. Consequently, the output at that location will have either a highly positive or a highly negative value.

### 1.A. Non-separable implementation (1 point)

In the next cell, we provide the function `filterNonSeparable()`. This function **is supposed to perform a convolution** on two input images, `img` and `mask`. However, whoever implemented the function (*it certainly wasn't us*) had not heard about the difference between a correlation and a convolution. To put it simply, the function does not yield the correct result. **For 1 point**, find the mistake and correct it in the cell below.

*Hints*:
- The two-dimensional convolution between two images `h` and `f` is given by
  $$(h \ast f)[x,y] = \sum_{m \in \mathbb{Z}}\sum_{n \in \mathbb{Z}}f[m,n]h[x-m,y-n]$$
- There are a few considerations that you will need to keep in mind when going from formula to code. For example, in the formula above, $x-m$ is negative for some values, but the indices of the JavaScript Array are all positive.

In [ ]:
%use javascript

// function that performs a convolution on the two input parameters img and mask
function filterNonSeparable(img, mask){
    // create output Image, of the same shape as the input
    var output = new Image(img.shape());
    // iterate through every pixel
    for(var x = 0; x < img.nx; x++){  
        for(var y = 0; y < img.ny; y++){
            // get neighbourhood with the size of mask of current position
            var neigh = img.getNbh(x, y, mask.nx, mask.ny)
            // perform convolution
            var val = 0;
            for(var k = 0; k < mask.nx; k++){
                for(var l = 0; l < mask.ny; l++){
                    val += neigh.getPixel(k, l) * mask.getPixel(k, l);
                }
            }
            // assign value on output image
            output.setPixel(x, y, val)
        }
    }
    return output
}

Run the next cell to perform a sanity check (convolution of the mask with an impulse image).

In [ ]:
// Lets do a sanity check
// we use a simple 3x3 impulse image
var impulse = new Image([[0, 0, 0], [0, 1, 0], [0, 0, 0]]);
// declare mask
var mask = new Image([[-1, -2, -1],[0, 0, 0],[1, 2, 1]])
var nonsymmetric_mask = new Image([[1, 2, 3],[4, 5, 6],[7, 8, 9]]);
// check the result
console.log('Convolving by an impulse should return the unchanged mask.\nMask:\n' + mask.visualize() )
console.log('\n' + nonsymmetric_mask.visualize())
console.log('Symmetric Result:\n' + filterNonSeparable(impulse, mask).visualize());
console.log('Nonsymmetric Result:\n' + filterNonSeparable(impulse, nonsymmetric_mask).visualize());
if(filterNonSeparable(impulse, mask).imageCompare(mask) == false){
    console.log('\nWARNING!!!\nThe function `filterNonSeparable` still provides an incorrect output.\n')
}else{
    console.log('It seems that the function works with a symmetric mask.')
    if (filterNonSeparable(impulse, nonsymmetric_mask).imageCompare(nonsymmetric_mask) == false){
        console.log('\nWARNING!!!\nIt seems that the function does not work with a nonsymmetric mask however.\n')
    }
    else{
        console.log('It seems that the function also works with a nonsymmetric mask')
    }
}

#### Applying the function

Run the next cell to apply the `filterNonSeparable` function with the mask given above to the images `bikesgray`, `camera`, and `spots`. 

In [ ]:
%use javascript
%get bikesgray camera spots

// apply Non separable filter
var hedge_bike_nons   = filterNonSeparable(new Image(bikesgray), mask).toArray();
var hedge_spots_nons  = filterNonSeparable(new Image(spots),     mask).toArray();
var hedge_camera_nons = filterNonSeparable(new Image(camera),    mask).toArray();

#### Visualize the results
In the next cell, we will visualize the results of edge detection. Run the next cell to visualize the results and use the `Next` / `Prev` buttons to cycle through the images.

In [ ]:
%use sos
%get hedge_bike_nons hedge_camera_nons hedge_spots_nons --from javascript

# Declare parameters for viewer
image_list_nons = [hedge_bike_nons, hedge_camera_nons, hedge_spots_nons]
title_list = ['Bike - Horizontal edges', 'Camera - Horizontal edges', 'Spots - Horizontal edges']

# Make sure that the images are numpy arrays and not lists
for image in range(len(image_list_nons)):
    image_list_nons[image] = np.array(image_list_nons[image])

vedge_nons_viewer = viewer(image_list_nons, title=title_list, widgets=True)

### 1.B. Separable implementation (2 points)

In this subsection, you will modify the `filterSeparable()` function that we provided to implement the separable version of the filter. This function is based on the `filter1D()` function (also declared in the next cell), which takes as input a row/column of the image and a 1D mask, and performs a 1D convolution. Recall that separable filters are divided into two 1D masks (`mx` and `my` in the code below). 

In the next cell, **for 1 point**, complete the function `filterSeparable()` by implementing the column filtering. The row filtering is provided and can be used as a reference.
    
**Note**: `img.getRow(y)` extracts the $y^{\text{th}}$ row from `img` while `output.putRow(y, new_row)` inserts `new_row` into the $y^{\text{th}}$ row of `output`. Similarly, you can use `.getColumn(x)` and `.putColumn(x, new_column)` to extract and insert columns from/to `Image` objects.

*Hint*: If you don't remember how a separable filter works, review the theory in your course notes.

In [ ]:
%use javascript

// function that performs a separable convolution on img using two 1D masks mx and my
function filterSeparable(img, mx, my){
    // initialize output Image, of the same shape as the input
    var output = new Image(img.shape());
    
    // iterate through every row 
    for(var y = 0; y < img.ny; y++){
        // extract row
        var row = img.getRow(y);
        // apply filter
        var new_row = filter1D(row, mx)
        // set column in output variable
        output.putRow(y, new_row);    
    }
    // iterate through every column
    for(var x = 0; x < img.nx; x++){
        // filter the columns
        
        // extract column, apply filter and set
        // YOUR CODE HERE
        
    }
    return output
}

// function that applies a 1D filter
function filter1D(img, mask){
    // transpose the input variables if necessary
    if(img.nx == 1){
        img.transposeImage();
    }
    if(mask.nx == 1){
        mask.transposeImage();
    }
    // create the output image
    var output = new Image(img.shape());
    // iterate through all pixels
    for(var x = 0; x < img.nx; x++){
        // get the neighbourhood around position x
        var neigh = img.getNbh(x, 0, mask.nx, 1);
        // declare a variable to store the values of the convolution. 
        var val = 0;
        // iterate through the neighbourhood
        for(var i = 0; i < neigh.nx; i++){
            // perform convolution
            val += neigh.getPixel(i, 0) * mask.getPixel(mask.nx - 1 - i, 0);
        }
        // set value in output array
        output.setPixel(x, 0, val);
    }
    return output
}

#### 1D masks

Now, before testing your separable implementation and **for 1 point** change the masks `mask_x` and `mask_y` in the cell below to the correct 1D masks corresponding to the separable version of the horizontal-edge detection filter presented [at the beginning of this section](#1.-Edge-detection-(3-points)).

In [ ]:
%use javascript
%put mask_x mask_y
// change the masks to the correct value 
var mask_x = [0, 0, 0];
var mask_y = [0, 0, 0];

// YOUR CODE HERE

// convert the masks to Image objects
var mask_x_img = new Image([mask_x]);
var mask_y_img = new Image([mask_y]);

Now, run the next cell for a quick sanity check on your `filterSeparable` function and on your 1D masks. We will convolve an impulse with your $1$D masks, and the result should be exactly the filter that we presented [at the beginning of the section](#1.-Edge-detection-(3-points)).

In [ ]:
%use javascript
// again let's do a sanity check with an impulse image
var impulse = new Image([[0, 0, 0], [0, 1, 0], [0, 0, 0]]);
// declare correct 2D mask
var mask = new Image([[-1, -2, -1], [0, 0, 0], [1, 2, 1]]);

// First we check your masks with a low tolerance
if(filterSeparable(impulse, mask_x_img, mask_y_img).imageCompare(mask, tol=1e-10) == false){
    // Repeat with high tolerance
    if(filterSeparable(impulse, mask_x_img, mask_y_img).imageCompare(mask, tol=1e-4) == false){
    console.log('WARNING!!!\nSorry it looks like the separable filter or the 1D masks are not yet correct.');
    }else{
        console.log('WARNING!!!\nYour masks are almost correct, but the accuracy does not seem quite enough...');
}}else{
    console.log('The separable filter and the 1D masks have passed this sanity check!');
}

Now run the next cell to apply the function to the same three images we have been using before.

In [ ]:
%use javascript
%get bikesgray spots camera
// Apply function on previously defined Images
var hedge_bike_s   = filterSeparable(new Image(bikesgray), mask_x_img, mask_y_img).toArray();
var hedge_spots_s  = filterSeparable(new Image(spots),     mask_x_img, mask_y_img).toArray();
var hedge_camera_s = filterSeparable(new Image(camera),    mask_x_img, mask_y_img).toArray();

Run the next one to visualize the results.

In [ ]:
%use sos
%get hedge_bike_s hedge_camera_s hedge_spots_s --from javascript
# Declare the image list
image_list_sep = [hedge_bike_s, hedge_camera_s, hedge_spots_s]
title_list = ['Bike - Horizontal edges', 'Camera - Horizontal edges', 'Spots - Horizontal edges']

# Make sure that the images are numpy arrays and not lists
for i, image in enumerate(image_list_sep):
    image_list_sep[i] = np.array(image)

plt.close('all')
vedge_s_viewer = viewer(image_list_sep, title=title_list, widgets=True)

### 1.C. Comparison: separable vs non-separable

Did you notice any difference between the separable and non-separable versions of the results? Did you expect any difference? 

Explore the two histograms and zoom into different areas of the three images to see if they are truly equal. There are several tests you can do to measure the equality of both implementations. You can even take the difference between the two images and visualize it (`image1 - image2`), or you can use the different functions provided by NumPy to compare arrays (see [numpy.testing.assert_array_equal](https://numpy.org/doc/stable/reference/generated/numpy.testing.assert_array_equal.html) and [numpy.testing.assert_array_almost_equal](https://numpy.org/doc/stable/reference/generated/numpy.testing.assert_array_almost_equal.html)). Use the next blank cell to do any necessary exploration. 

*Hint*: If both your implementations are correct, the difference should be negligible.

In [ ]:
%use sos

# YOUR CODE HERE

### 1.D. Edge detection in Python

In Python, the two functions `filterNonSeparable` and `filterSeparable` you implemented above are provided by the [`scipy.ndimage`](https://docs.scipy.org/doc/scipy/reference/ndimage.html) library. They are called [`convolve`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.ndimage.convolve.html) and [`convolve1d`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.ndimage.convolve1d.html). 

The arguments of this function are:

 * `input`: the original image
 * `weights`: the mask to convolve the image with
 * `axis` **(only for `convolve1d`)**: The axis along which the 1D filter will be applied ($0$ to filter columns, $1$ for rows)
 * `output`: array or dtype (see technical note below)
 * `mode`: specifies the boundary conditions (see documentation for options). Defaults to `'reflect'`, which is the same we recommend and use in the course, as well as the one implemented in the JavaScript class `ImageAccess`.

**Examples:**

To apply a non-separable filter with a 2D mask `mask` to the image `img`, we use:
```python
filtered_img = ndi.convolve(img, mask)
```
To apply a separable filter with two 1D masks, `mask_x` and `mask_y`, to the image `img`, we use:
```python
# filter the columns
filtered_img = ndi.convolve1d(img, mask_y, axis=0)
# filter the rows
filtered_img = ndi.convolve1d(filtered_img, mask_x, axis=1)
```

**Note**: 
We have been using `float64` (64-bit floating point) as the `dtype` (which stands for *data type*) of the images (see the [Setup](#Setup) section). This `dtype` is the most versatile. However, it is also very common to store images and work with them as `np.uint8` (8-bit unsigned integer, which spans the range $[0, 255]$).

If you need to work with integers at some point (e.g., to save memory or increase performance), you can choose `np.int32` (32-bit signed integers) since the result will still be an integer because of the integer mask. 
    
The `dtype` is something that you will have to keep in mind constantly when working with images, and that can be the source of many bugs.

Run the next cell to compare the Python implementations to the JavaScript implementations, and make sure that you get the same result. If you did everything correctly in Sections [1.A.](#1.A.-Non-separable-implementation-(1-point)) and [1.B.](#1.B.-Separable-implementation-(2-points)), this cell will display a winning message. Otherwise, it will plot your images **with the regions that differ from the correct answer highlighted in red**, which should give you a clue on where you might have gone wrong.

In [ ]:
%use sos 
# Define the 2D mask
mask = np.array([[-1, -2, -1],[0, 0, 0],[1, 2, 1]])
# Apply non-separable Filter
hedge_ndi = ndi.convolve(bikesgray, mask, output=np.int32)
# Apply separable Filter - we use mask_x and mask_y you defined in the exercise before
# filter the columns
hedge_ndi_s = ndi.convolve1d(bikesgray, mask_y, axis=0, output=np.float64)
# filter the rows
hedge_ndi_s = ndi.convolve1d(hedge_ndi_s, mask_x, axis=1, output=np.float64)

# Visualize if necessary
plt.close('all')
if np.allclose(hedge_ndi, hedge_bike_nons):
    print('Your non-separable implementation is correct!')
else :
    print('Review your non-separable JS implementation!')
    viewer([np.array(hedge_bike_nons), hedge_ndi], title=['JS non-separable', 'Python non-separable'], compare=True)

if np.allclose(hedge_ndi_s, hedge_bike_s):
    print('Congratulations!\nYour separable filter is correct!')
else :
    print('Review your separable JS implementation. \
           \nThe problem could be either on your 1D masks, or on the implementation itself. The previous sanity checks and the red areas should guide you.')
    viewer([np.array(hedge_bike_s), hedge_ndi_s], title=['JS separable', 'Python separable'], compare=True)

# hedge_viewer = viewer([np.array(hedge_bike_nons), hedge_ndi, np.array(hedge_bike_s), hedge_ndi_s], title=['JS non-separable', 'Python non-separable', 'JS separable', 'Python separable'], subplots=(2,2))

## 2. Implementation and classification of digital filters (6 points)

In this exercise, we will practice implementing and classifying masks or digital filters. Moreover, you will practice separable and non-separable implementations in Python. We will look at $6$ different digital filters. Your task, **for 6 points**, is to implement and apply each of the filters in Python (**0.5** points per filter) to an image, and classify their visual effect (**0.5** points per filter).

Note that you can define a 2D mask in Python with 
```python
mask = np.array([[ 1,  2,  3,  4,  5], 
                 [ 6,  7,  8,  9, 10], 
                 [11, 12, 13, 14, 15],
                 [16, 17, 18, 19, 20],
                 [21, 22, 23, 24, 25]])
```
and define a 1D mask in Python with
```python
mask = np.array([1, 2, 3, 4, 5])
```

**Note**: If a 2D mask is separable, it is best practice to implement it as a separable filter using `ndi.convolve1d` with the appropriate 1D masks (that you will have to find). Otherwise, use `ndi.convolve`. Review Section 1.D for an example. **If a mask is separable and you do not implement it as such, you will not get any points.** 

**Notes on the separability of filters:**

Some filters, in particular, simple edge detector filters, can be recognized as separable intuitively. Furthermore, their 1D masks can also be intuitively proposed. From these 1D masks, a simple test consists of taking the matrix product between them, which should result in the original filter. However, a more systematic approach is available.

As you should know, any matrix $\mathbf{\mathrm{M}}\in\mathbb{R}^{m\times n}$ can be factorized into its components $\mathbf{\mathrm{USV^{T}}}$ by Singular Value Decomposition, where $\mathbf{\mathrm{U}}\in\mathbb{R}^{m\times m}$, $\mathbf{\mathrm{S}}\in\mathbb{R}^{\min(m, n)\times \min(m, n)}$ corresponds to a diagonal matrix with the singular values of $\mathbf{\mathrm{M}}$, and $\mathbf{\mathrm{V}}\in\mathbb{R}^{n\times n}$. It follows from the definition of the matrix product that if only one singular value is nonzero, the filter is separable.

Nonetheless, for simple filters, we still recommend using the *intuitive* filters. If you want to extract the filters using SVD, you can use the [`svd`](https://numpy.org/doc/stable/reference/generated/numpy.linalg.svd.html) function of the NumPy Linear Algebra package. We leave you the task of extracting the actual $1\mathrm{D}$ masks from it.

### 2.A. Mask A (1 point)

In the cell below, **For 0.5 points**, implement the function `filter_A`, which convolves an image given as input parameter with the filter

```python3
    [ 0,  0,  0,  0,  0]
    [ 0,  0,  0,  0,  0]
A = [ 1,  1,  1,  1,  1]
    [ 0,  0,  0,  0,  0]
    [ 0,  0,  0,  0,  0]
```

Remember to check if the filter is separable, and if so, implement it accordingly.

**Note**: Hard-code the mask inside the function. That means, do not take the mask from a local variable, nor as an input parameter of the function `filter_A` (the only input parameter is an image), but declare the mask **inside** the function. This is true for all the following six exercises.

In [ ]:
%use sos

# Function that filters img with the mask A
def filter_A(img):
    output = img.copy()
    
    # Filter the image with mask A    
    # YOUR CODE HERE
    
    return output

Run the next cell to visualize the result of applying `filter_A` to the image `bikesgray`.

*Hint*: If you cannot see the whole `viewer`, remember that you can use `Ctrl b` to hide the sidebar on the left. You can also use `Ctrl -` to zoom out a bit.

In [ ]:
%use sos
# Apply the function
bikesgray_A = filter_A(bikesgray)
# Visualize the result
plt.close('all')
vis_A = viewer([bikesgray, bikesgray_A], title=['Original', 'Filtered with mask A'], subplots=(1,2))

**For 0.5 points** answer the following MCQ:

* Which of the following keywords best describes the visual effect of this filter?

1. isotropic blurring, or
2. vertical blurring, or
3. horizontal blurring, or
4. diagonal blurring, or
5. sharpening, or
6. vertical-edge detection, or
7. horizontal-edge detection, or
8. identity, or
9. Laplacian: a second order derivative to find edges (maxima and minima), or
10. [embossing](https://en.wikipedia.org/wiki/Image_embossing), or difference directional filter.

In [ ]:
%use sos
# Assign your answer to this variable
answer = None
# YOUR CODE HERE

In [ ]:
%use sos
# Sanity check
if not answer in list(range(1,11)):
    print('WARNING!\nAnswer one of 1, 2, 3, 4, 5, 6, 7, 8, 9 or 10.')

### 2.B. Mask B (1 point)

In the cell below, **For 0.5 points**, implement the function `filter_B`, which convolves the input image with the filter

```python3
    [  -1,  -4,  -6,  -4,  -1]
    [  -4, -16, -24, -16,  -4]
B = [  -6, -24, 476, -24,  -6]
    [  -4, -16, -24, -16,  -4]
    [  -1,  -4,  -6,  -4,  -1]
```

Remember to check if the filter is separable, and in that case, implement it in that way.

In [ ]:
%use sos

# Function that filters img with the mask B
def filter_B(img):
    output = img.copy()
    
    # Filter the image with mask B    
    # YOUR CODE HERE
    
    return output

Run the next cell to visualize the result of applying `filter_B` to the image `bikesgray`.

In [ ]:
%use sos
# Apply the function
bikesgray_B = filter_B(bikesgray)
# Visualize the result
plt.close('all')
vis_B = viewer([bikesgray, bikesgray_B], title=['Original', 'Filtered with mask B'], subplots=(1,2))

**For 0.5 points** answer the following MCQ:

* Which of the following keywords best describes the visual effect of this filter?

1. isotropic blurring, or
2. vertical blurring, or
3. horizontal blurring, or
4. diagonal blurring, or
5. sharpening, or
6. vertical-edge detection, or
7. horizontal-edge detection, or
8. identity, or
9. Laplacian: a second order derivative to find edges (maxima and minima), or
10. [embossing](https://en.wikipedia.org/wiki/Image_embossing), or difference directional filter.

In [ ]:
%use sos
# Assign your answer to this variable
answer = None
# YOUR CODE HERE

In [ ]:
%use sos
# Sanity check
if not answer in list(range(1,11)):
    print('WARNING!\nAnswer one of 1, 2, 3, 4, 5, 6, 7, 8, 9 or 10.')

### 2.C. Mask C (1 point)

In the cell below, **For 0.5 points**, implement the function `filter_C`, which convolves the input image with the filter

```python3
    [ -2,  0, -1,  0,  0]
    [  0, -2, -1,  0,  0]
C = [ -1, -1,  1,  1,  1]
    [  0,  0,  1,  2,  0]
    [  0,  0,  1,  0,  2]
```

Remember to check if the filter is separable, and in that case, implement it in that way.

In [ ]:
%use sos

# Function that filters img with the mask C
def filter_C(img):
    output = img.copy()
    
    # Filter the image with mask C
    # YOUR CODE HERE
    
    return output

Run the next cell to visualize the result of applying `filter_C` to the image `bikesgray`.

In [ ]:
%use sos
# Apply the function
bikesgray_C = filter_C(bikesgray)
# Visualize the result
plt.close('all')
vis_C = viewer([bikesgray, bikesgray_C], title=['Original', 'Filtered with mask C'], subplots=(1,2))

**For 0.5 points** answer the following MCQ:

* Which of the following keywords best describes the visual effect of this filter?

1. isotropic blurring, or
2. vertical blurring, or
3. horizontal blurring, or
4. diagonal blurring, or
5. sharpening, or
6. vertical-edge detection, or
7. horizontal-edge detection, or
8. identity, or
9. Laplacian: a second order derivative to find edges (maxima and minima), or
10. [embossing](https://en.wikipedia.org/wiki/Image_embossing), or difference directional filter.

In [ ]:
%use sos
# Assign your answer to this variable
answer = None
# YOUR CODE HERE

In [ ]:
%use sos
# Sanity check
if not answer in list(range(1,11)):
    print('WARNING!\nAnswer one of 1, 2, 3, 4, 5, 6, 7, 8, 9 or 10.')

### 2.D. Mask D (1 point)

In the cell below, **For 0.5 points**, implement the function `filter_D`, which convolves the input image with the filter

```python3
    [ -1, -2,  0,  2,  1]
    [ -1, -2,  0,  2,  1]
D = [ -1, -2,  0,  2,  1]
    [ -1, -2,  0,  2,  1]
    [ -1, -2,  0,  2,  1]
```

Remember to check if the filter is separable, and in that case, implement it in that way.

In [ ]:
%use sos

# Function that filters img with the mask D
def filter_D(img):
    output = img.copy()
    
    # Filter the image with mask D
    # YOUR CODE HERE
    
    return output

Run the next cell to visualize the result of applying `filter_D` to the image `bikesgray`.

In [ ]:
%use sos
# Apply the function
bikesgray_D = filter_D(bikesgray)
# Visualize the result
plt.close('all')
vis_D = viewer([bikesgray, bikesgray_D], title=['Original', 'Filtered with mask D'], subplots=(1,2))

**For 0.5 points** answer the following MCQ:

* Which of the following keywords best describes the visual effect of this filter?

1. isotropic blurring, or
2. vertical blurring, or
3. horizontal blurring, or
4. diagonal blurring, or
5. sharpening, or
6. vertical-edge detection, or
7. horizontal-edge detection, or
8. identity, or
9. Laplacian: a second order derivative to find edges (maxima and minima), or
10. [embossing](https://en.wikipedia.org/wiki/Image_embossing), or difference directional filter.

In [ ]:
%use sos
# Assign your answer to this variable
answer = None
# YOUR CODE HERE

In [ ]:
%use sos
# Sanity check
if not answer in list(range(1,11)):
    print('WARNING!\nAnswer one of 1, 2, 3, 4, 5, 6, 7, 8, 9 or 10.')

### 2.E. Mask E (1 point)

In the cell below, **For 0.5 points**, implement the function `filter_E`, which convolves the input image with the filter 

```python3
    [  0,  0,  1,  0,  0]
    [  0,  1,  2,  1,  0]
E = [  1,  2,-16,  2,  1]
    [  0,  1,  2,  1,  0]
    [  0,  0,  1,  0,  0]
```
Remember to check if the filter is separable, and in that case, implement it in that way.

In [ ]:
%use sos

# Function that filters img with the mask E
def filter_E(img):
    output = img.copy()
    
    # Filter the image with mask E
    # YOUR CODE HERE
    
    return output

Run the next cell to visualize the result of applying `filter_E` to the image `bikesgray`.

In [ ]:
%use sos
# Apply the function
bikesgray_E = filter_E(bikesgray)
# Visualize the result
plt.close('all')
vis_E = viewer([bikesgray, bikesgray_E], title=['Original', 'Convolved with filter E'], subplots=(1,2))

**For 0.5 points** answer the following MCQ:

* Which of the following keywords best describes the visual effect of this filter?

1. isotropic blurring, or
2. vertical blurring, or
3. horizontal blurring, or
4. diagonal blurring, or
5. sharpening, or
6. vertical-edge detection, or
7. horizontal-edge detection, or
8. identity, or
9. Laplacian: a second order derivative to find edges (maxima and minima), or
10. [embossing](https://en.wikipedia.org/wiki/Image_embossing), or difference directional filter.

In [ ]:
%use sos
# Assign your answer to this variable
answer = None
# YOUR CODE HERE

In [ ]:
%use sos
# Sanity check
if not answer in list(range(1,11)):
    print('WARNING!\nAnswer one of 1, 2, 3, 4, 5, 6, 7, 8, 9 or 10.')

### 2.F. Mask F (1 point)

In the cell below, **For 0.5 points**, implement the function `filter_F`, which convolves the input image with the filter 

```python3
    [0, 0, 0, 0, 0]
    [0, 0, 0, 0, 0]
F = [0, 0, 1, 0, 0]
    [0, 0, 0, 0, 0]
    [0, 0, 0, 0, 0]
```

Remember to check if the filter is separable, and in that case, implement it in that way.

In [ ]:
%use sos

# Function that filters img with the mask F
def filter_F(img):
    output = img.copy()
    
    # Filter the image with mask F
    # YOUR CODE HERE
    
    return output

Run the next cell to visualize the result of applying `filter_F` to the image `bikesgray`.

In [ ]:
%use sos
# Apply the function
bikesgray_F = filter_F(bikesgray)
# Visualize the result
plt.close('all')
vis_F = viewer([bikesgray, bikesgray_F], title=['Original', 'Filtered with mask F'], subplots=(1,2))

**For 0.5 points** answer the following MCQ:

* Which of the following keywords best describes the visual effect of this filter?

1. isotropic blurring, or
2. vertical blurring, or
3. horizontal blurring, or
4. diagonal blurring, or
5. sharpening, or
6. vertical-edge detection, or
7. horizontal-edge detection, or
8. identity, or
9. Laplacian: a second order derivative to find edges (maxima and minima), or
10. [embossing](https://en.wikipedia.org/wiki/Image_embossing), or difference directional filter.

In [ ]:
%use sos
# Assign your answer to this variable
answer = None
# YOUR CODE HERE

In [ ]:
%use sos
# Sanity check
if not answer in list(range(1,11)):
    print('WARNING!\nAnswer one of 1, 2, 3, 4, 5, 6, 7, 8, 9 or 10.')

🎉 Congratulations on finishing the first part of the digital filtering lab!

Make sure to save your notebook (you might want to keep a copy on your personal computer) and upload it to [Moodle](https://moodle.epfl.ch/course/view.php?id=522), in a zip file with the other notebook of this lab.

* Keep the name of the notebook as: *1_filtering.ipynb*,
* Name the zip file: *filtering_lab.zip*.